In [ ]:
from deck import BlackjackShoe
from dealer import Dealer
from round import BlackjackRound
from hand import Hand
from game import Game, BLACKJACKSIXTOFIVEPAYOUT, BLACKJACKTHREETOTWOPAYOUT
from strategies.strategy import StrategyTable
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
rounds = []
rounds_count_dict = {}
games = []
num_players = 1
num_games = 1
bet_size = 100
num_rounds = 10000000
for _ in range(num_games):
    game = Game(2, num_players=num_players, strategy=StrategyTable["MULTIDECK"], hit_on_soft_17=True, blackjack_payout=BLACKJACKSIXTOFIVEPAYOUT, min_bet=bet_size, denominations=100, player_bankroll=0, resplit_till=4)
    round_data, round_count_data = game.play(num_rounds, print_cards=False, print_round_results=False)
    rounds.extend(round_data)
    rounds_count_dict = round_count_data
    games.append(game.house_bankroll)

profit_sum = 0
wins = 0
losses = 0
pushes = 0
for g in rounds:
    # print(rounds_count)
    if g > 0:
        wins += 1
    elif g < 0:
        losses += 1
    else:
        pushes += 1
    profit_sum += g
print(profit_sum)

print(profit_sum / num_players / num_games / num_rounds)
print(f"Wins: {wins / num_rounds}, Losses: {losses / num_rounds}, Pushes: {pushes / num_rounds}")
print()

In [ ]:
rounds_count_edge_dict = {}
for count, round_profits in rounds_count_dict.items():
    num_rounds = len(round_profits)
    num_players = 1
    num_games = 1
    profits = sum(round_profits)
    rounds_count_edge_dict[(count, num_rounds)] = profits / num_rounds / num_players / num_games

for count, edge in sorted(rounds_count_edge_dict.items()):
    print(f"Count: {count}, Edge: {edge}")

In [ ]:
plt.hist(rounds, bins=20, facecolor="blue", alpha=0.5)

plt.show()

In [ ]:
plt.hist(games, bins=20, facecolor="blue", alpha=0.5)
# sns.kdeplot(games)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert house bankroll changes (games list) into profits per game
house_profits = np.array(games)  # Assuming games stores house profit for each game

# Calculate statistics
mean_house_profit = np.mean(house_profits)
ci_lower = np.percentile(house_profits, 2.5)
ci_upper = np.percentile(house_profits, 97.5)

# Plot histogram of actual house profits
plt.figure(figsize=(10, 6))
plt.hist(house_profits, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(mean_house_profit, color='red', linestyle='dashed', linewidth=2, label=f'Mean: ${mean_house_profit:,.2f}')
plt.axvline(ci_lower, color='green', linestyle='dashed', linewidth=2, label=f'95% CI Lower: ${ci_lower:,.2f}')
plt.axvline(ci_upper, color='green', linestyle='dashed', linewidth=2, label=f'95% CI Upper: ${ci_upper:,.2f}')
plt.title(f'House Profit Distribution Over {num_games} Games ({num_players} Player(s), ${bet_size} Bet)')
plt.xlabel('House Profit')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Print results
print(f"Expected House Profit: ${mean_house_profit:,.2f}")
print(f"95% Confidence Interval: (${ci_lower:,.2f}, ${ci_upper:,.2f})")


In [ ]:
import numpy as np

# Example: house_wins is an array of house profit per round from your simulation.
# For illustration, let's assume you already simulated these values.
# house_wins = np.array([...])
house_wins = np.array(rounds)
n = len(house_wins)
sample_mean = np.mean(house_wins)
sample_std = np.std(house_wins, ddof=1)


In [ ]:
from scipy import stats

# Hypothesized mean (e.g., house edge is $2.50 per round)
mu0 = 0.41349 * 1

# Calculate the t statistic
t_statistic = (sample_mean - mu0) / (sample_std / np.sqrt(n))

# Degrees of freedom is n-1
df = n - 1

# Two-tailed p-value
p_value = 2 * (1 - stats.t.cdf(np.abs(t_statistic), df=df))

print(f"Sample Mean: {sample_mean:.4f}")
print(f"t-statistic: {t_statistic:.4f}")
print(f"p-value: {p_value:.4f}")
